In [ ]:
# 엑스레이 이미지 파일 변환
import shutil # 파일, 폴더 복사 모듈
import cv2 # openCV 모듈 
import os
import numpy as np
import pandas as pd
import natsort # 숫자로 된 파일명 정렬 모듈
from PIL import Image

df = pd.read_excel('C:\\Users\\user\\Desktop\\test.xlsx', header=None) # 엑셀 파일 위치 지정

dir_working = 'C:\\Users\\user\\Desktop\\20220503' # 촬영 데이터가 모여져 있는 폴더 경로

i = 0 # 파일 명 찾는 가변 변수
j = 0 # Serial Number 붙일 때 활용하는 가변 변수
height = 2500
width = 896

dir_working_copy = dir_working + '\\이미지 변환'
dir_working_result = dir_working + '\\result'

if not os.path.exists(dir_working_copy):
    shutil.copytree(dir_working, dir_working_copy) # 원본 폴더(dir_working)는 보관용으로 남기고, copy 폴더를 생성하여 작업.
if not os.path.exists(dir_working_result):
    os.mkdir(dir_working_result)

filelist = os.listdir(dir_working_copy)
print(filelist)

for file in filelist:

    os.chdir(dir_working_copy) # change directory
    
    # read image as binary
    read_binary = open(dir_working_copy+'/'+file, "rb") # 1. 파일 경로, 2. read + binary (rb)
    img = np.fromfile(read_binary, dtype='uint16', sep="", count=(height*width))
    read_binary.close()

    img = np.reshape(img, [height, width]) # 1500 -> height, 896 -> width
    
    # image 저장
    markfile = df.values[i][1] + "_" + str(j) + '.png'
    
    cv2.imwrite(markfile, img)

    print(file)
    print(markfile)
    print(df.values[i][1])

    # image detection
    img_detection = cv2.imread(markfile, cv2.IMREAD_COLOR)

    # 이미지 그레이 변환
    gray = cv2.cvtColor(img_detection, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, ksize=(5,5), sigmaX=0)

    redlalwlt, th1 = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY_INV)
    binary = cv2.bitwise_not(th1)

    contours, hierarchy = cv2.findContours(th1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contours_xy = np.array(contours)

    value = list()

    for x in range(len(contours_xy)) :
        for y in range(len(contours_xy[x])):
            value.append(contours_xy[x][y][0][1])

    y_min = min(value)

    print(y_min)

    if (y_min - 65 <= 0):
        y_min = 0
    else:
        y_min = y_min - 65
    
    img = img[y_min:y_min + 760, :]

    
    # 이미지 변환이 끝난 raw 파일은 필요 없으므로 삭제함.
    if os.path.exists(dir_working_copy + '/' + file):
        os.remove(dir_working_copy +'/'+ file)
    
    os.chdir(dir_working_result)
    cv2.imwrite(markfile, img)
    

    # Serial Number 1 증가  
    j = j + 1

    # 9-view
    if (j == 9):
        i = i + 1
        j = 0

In [ ]:
# 트레이 디텍션 (엑스레이 이미지)

import cv2
import matplotlib.pyplot as plt
import numpy as np

# 이미지 불러오기
img = cv2.imread('a_4.png', cv2.IMREAD_COLOR)

print(img)

# 이미지 그레이 변환
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_canny = cv2.Canny(gray, 50, 150)
blur = cv2.GaussianBlur(gray, ksize=(5,5), sigmaX=0)

redlalwlt, th1 = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY_INV)
binary = cv2.bitwise_not(th1)

contours, hierarchy = cv2.findContours(th1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
contours_xy = np.array(contours)

value = list()

for i in range(len(contours_xy)) :
    for j in range(len(contours_xy[i])):
        value.append(contours_xy[i][j][0][1])

y_min, y_max = min(value), max(value)

if (max(value) > 800):
    y_max = 800


print(y_max, y_min)

for i in range(len(contours)):
    cv2.drawContours(img, [contours[i]], 0, (0, 0, 255), 2)
    
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Canny 윤곽선 검출
import cv2

img_gray = cv2.imread('a_5.png', cv2.IMREAD_GRAYSCALE)
cv2.imshow("Original", img_gray)

img_canny = cv2.Canny(img_gray, 50, 150)
cv2.imshow("Canny Edge", img_canny)

print(img_canny)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 트랙바
def nothing():
    pass

img_gray = cv2.imread('a_2.png', cv2.IMREAD_GRAYSCALE)

cv2.namedWindow("Canny Edge")
cv2.createTrackbar('low threshold', 'Canny Edge', 0, 1000, nothing)
cv2.createTrackbar('high threshold', 'Canny Edge', 0, 1000, nothing)

cv2.setTrackbarPos('low threshold', 'Canny Edge', 50)
cv2.setTrackbarPos('high threshold', 'Canny Edge', 150)

cv2.imshow("Original", img_gray)

while True:
    low = cv2.getTrackbarPos('low threshold', 'Canny Edge')
    high = cv2.getTrackbarPos('high threshold', 'Canny Edge')

    img_canny = cv2.Canny(img_gray, low, high)
    cv2.imshow("Canny Edge", img_canny)

    if cv2.waitKey(1)&0xFF == 27:
        break

cv2.destroyAllWindows()